In [97]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By

In [104]:
#-------------------Extract-------------------------
urls = ["https://forecast.weather.gov/MapClick.php?lat=57.0826&lon=-135.2692#.Y-vs_9LMJkg",
        'https://forecast.weather.gov/MapClick.php?lat=45.5118&lon=-122.6756#.Y-vtHNLMJkg',
        'https://forecast.weather.gov/MapClick.php?lat=40.7143&lon=-74.006#.Y-vtQtLMJkg'
        ]
        
combined_df = pd.DataFrame()

for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #various containers
    item1 = soup.find_all(id='current_conditions-summary')
    item2 = soup.find_all(id='current_conditions_detail')
    item3 = soup.find_all(id='tombstone-container')
    item4 = soup.find_all(id='tombstone-container')

    #raw data
    temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
    temp_min = soup.find('p', {'class': 'temp temp-low'}).text.strip()
    temp_max = soup.find('p', {'class': 'temp temp-high'}).text.strip()


    #df of temperatures
    df_temperature = pd.DataFrame({"temp" : temp_f,'tempmin': temp_min,'tempmax': temp_max})

    #df_2 is a df of current conditions in detail
    table = soup.find_all('table')
    df_2 = pd.read_html(str(table))[0]
    df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

    #merge both dataframes
    temp_df=pd.concat([df_temperature,df_2],axis=1)

    #scrape lattitude, longitude, and elevation 
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

    #scrape name
    location = soup.find('h2', {'class': 'panel-title'}).text.strip()

    #add location, lat, long, and elev to source_df
    temp_df['elevation_ft'] = elev
    temp_df['latitude'] = lat
    temp_df['longitude'] = lon
    temp_df['location'] = location

    combined_df = pd.concat([temp_df, combined_df], ignore_index=True, sort=False)

display(combined_df)



,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,location
0,53°F,Low: 43 °F,High: 59 °F,30.13 in (1019.5 mb),18°F (-8°C),25%,14 Feb 4:51 pm EST,10.00 mi,51°F (11°C),W 6 mph,154,40.78,73.97,"New York City, Central Park (KNYC)"
1,41°F,Low: 27 °F,High: 43 °F,30.2 in (1022.69 mb),33°F (1°C),73%,14 Feb 01:53 PM PST,10.00 mi,38°F (3°C),N 4 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)"
2,36°F,Low: 28 °F,High: 31 °F,30.09 in (1019.0 mb),25°F (-4°C),64%,14 Feb 12:53 pm AKST,10.00 mi,29°F (-2°C),SE 10 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI)


In [99]:
URLS = ['https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA',
        "https://www.wunderground.com/history/daily/us/or/portland",
        'https://www.wunderground.com/history/daily/us/ak/sitka/PASI'
        ]

precip_df = pd.DataFrame()

for URL in URLS:
    
    #create selenium web driver
    driver = webdriver.Chrome(r"C:/home/reed/.cache/selenium/chromedriver/linux64/109.0.5414.74")
    driver.get(URL)
    sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    #closes the pop up add if it appears otherwise continues to scrape the precip value from the page
    try:
        driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div/div/div/a').click()
    except:
        precip = driver.find_element(By.XPATH,"/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[2]/tr/td[1]").text

    df = pd.DataFrame([precip], columns=['precip'])
    precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)

    #quits the selenium driver
    driver.quit()

source_df =pd.concat([combined_df,precip_df],axis=1)
display(source_df)

/home/reed/Desktop/epicodus/team-week3/reed-work/venvteamweek/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # This is added back by InteractiveShellApp.init_path()


,Temperature_F,Temperature_C,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,location,precip
0,53°F,12°C,30.13 in (1019.5 mb),18°F (-8°C),25%,14 Feb 4:51 pm EST,10.00 mi,51°F (11°C),W 6 mph,154,40.78,73.97,"New York City, Central Park (KNYC)",0.00
1,41°F,5°C,30.18 in (1022.01 mb),32°F (0°C),70%,14 Feb 01:20 PM PST,10.00 mi,37°F (3°C),NW 6 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",0.14
2,36°F,2°C,30.11 in (1019.5 mb),25°F (-4°C),64%,14 Feb 11:53 am AKST,10.00 mi,27°F (-3°C),E 14 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI),0.11


In [100]:
#-------------Transformations------------------------
source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Temperature_F  3 non-null      object
 1   Temperature_C  3 non-null      object
 2   Barometer      3 non-null      object
 3   Dewpoint       3 non-null      object
 4   Humidity       3 non-null      object
 5   Last update    3 non-null      object
 6   Visibility     3 non-null      object
 7   Wind Chill     3 non-null      object
 8   Wind Speed     3 non-null      object
 9   elevation_ft   3 non-null      object
 10  latitude       3 non-null      object
 11  longitude      3 non-null      object
 12  location       3 non-null      object
 13  precip         3 non-null      object
dtypes: object(14)
memory usage: 464.0+ bytes


In [ ]:

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import dateutil

# Split the 'location' column into separate 'city' and 'state' columns
df[['city', 'state']] = df['location'].str.split(', ', expand=True)

# Convert 'lat' and 'lon' columns to float type
df[['lat', 'lon']] = df[['lat', 'lon']].astype(float)

# Convert 'elev' column to int type
df['elev_ft'] = df['elev_ft'].astype(int)

# Extract the numeric part of the temperature string and convert it to int
df['temp_f'] = df['temperature'].str.extract('(\d+)').astype(int)

# Convert temperature to Celsius and add to new column 'temp_c'
df['temp_c'] = (df['temp_f'] - 32) * 5/9

# Round 'temp_c' to nearest integer and cast to int type
df['temp_c'] = df['temp_c'].round().astype(int)

# Convert 'humidity' column to float type
df['humidity'] = df['humidity'].str.extract('(\d+)', expand=False).astype(float) / 100

# Split wind speed values into components and convert speed to int type
df['wind_speed'] = df['wind_speed'].str.extract('(\d+)', expand=False).fillna(0).astype(int)

# Set any missing or non-numeric wind speed values to 0
df['wind_speed'] = df['wind_speed'].replace('Calm', 0)

# Convert 'barometer' column to float type, and convert inches to millibars
df['barometer'] = df['barometer'].apply(lambda x: float(x.split()[0]) * 33.8639 if 'in' in x and x != 'NA' else None)

# Round 'barometer' to two decimal places
df['barometer'] = df['barometer'].round(2)

# Split 'dewpoint' column into separate 'dewpoint_f' and 'dewpoint_c' columns
df[['dewpoint_f', 'dewpoint_c']] = df['dewpoint'].str.extract('(\d+).*?(\d+)', expand=True).astype(int)

# Convert 'vis_miles' column to float type
df['vis_miles'] = df['vis_miles'].str.extract('(\d+\.\d+|\d+)', expand=False).astype(float).round(2)

# Split 'wind_chill' column into separate 'wind_chill_f' and 'wind_chill_c' columns
df[['wind_chill_f', 'wind_chill_c']] = df['wind_chill'].str.extract('(\d+).*?(\d+)', expand=True).astype(float)

# Convert 'last_update' column to datetime type with the desired format and time zone
df['last_update'] = df['last_update'].apply(lambda x: dateutil.parser.parse(x, tzinfos={'CST': dateutil.tz.tzoffset(None, -21600)}))

# Convert 'last_update' column to UTC
df['last_update'] = df['last_update'].apply(lambda x: x.astimezone(dateutil.tz.tzutc()))

# Drop columns that were split into two values
df = df.drop(['temperature', 'dewpoint', 'wind_chill'], axis=1)

# Move 'city' and 'state' columns after 'location' column
city = df.pop('city')
state = df.pop('state')
temp_f = df.pop('temp_f')
temp_c = df.pop('temp_c')
df.insert(1, 'city', city)
df.insert(2, 'state', state)
df.insert(6, 'temp_f', temp_f)
df.insert(7, 'temp_c', temp_c)

# Move 'last_update' column to last position
last_update = df.pop('last_update')
df['last_update'] = last_update


print(df)